# shuffle 코드

In [24]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import shutil
import glob

In [7]:
# train 디렉토리 경로
directory = r"D:\rawdata_disease_resize\house\train\labels"

# 데이터프레임을 저장할 리스트
data_list = []

# 디렉토리 내의 파일들에 대해 반복
for filename in tqdm(os.listdir(directory)):
    if filename.endswith(".txt"):
        file_path = os.path.join(directory, filename)
        with open(file_path, "r") as file:
            # 파일의 첫 번째 값 읽어오기
            class_label = file.readline().split()[0]

            # .txt 앞 문자열 추출
            txt_prefix = os.path.splitext(filename)[0]
            
            # 데이터프레임에 추가
            data_list.append([txt_prefix, class_label])

# 데이터프레임 생성
df_train = pd.DataFrame(data_list, columns=["filename", "class"])
df_train

100%|██████████████████████████████████████████████████████████████████████████| 23502/23502 [00:06<00:00, 3449.73it/s]


,filename,class
0,V006_77_0_00_11_03_12_0_c01_20201013_0000_S01,0
1,V006_77_0_00_11_03_12_0_c01_20201013_0000_S01_1,0
2,V006_77_0_00_11_03_12_0_c01_20201013_0001_S01,0
3,V006_77_0_00_11_03_12_0_c01_20201013_0001_S01_1,0
4,V006_77_0_00_11_03_12_0_c01_20201013_0002_S01,0
...,...,...
23497,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0005,3
23498,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0006,3
23499,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0007,3
23500,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0008,3


In [8]:
# validation 디렉토리 경로
directory = r"D:\rawdata_disease_resize\house\validation\labels"

# 데이터프레임을 저장할 리스트
data_list = []

# 디렉토리 내의 파일들에 대해 반복
for filename in tqdm(os.listdir(directory)):
    if filename.endswith(".txt"):
        file_path = os.path.join(directory, filename)
        with open(file_path, "r") as file:
            # 파일의 첫 번째 값 읽어오기
            class_label = file.readline().split()[0]

            # .txt 앞 문자열 추출
            txt_prefix = os.path.splitext(filename)[0]
            
            # 데이터프레임에 추가
            data_list.append([txt_prefix, class_label])

# 데이터프레임 생성
df_val = pd.DataFrame(data_list, columns=["filename", "class"])
df_val

100%|████████████████████████████████████████████████████████████████████████████| 4698/4698 [00:01<00:00, 3151.24it/s]


,filename,class
0,V006_77_0_00_11_03_12_0_c22_20201130_0001_S01_2,0
1,V006_77_0_00_11_03_13_0_c04_20201223_0000_S01_2,0
2,V006_77_0_00_11_03_13_0_c05_20201222_0000_S01_2,0
3,V006_77_0_00_11_03_13_0_c05_20201222_0001_S01_2,0
4,V006_77_0_00_11_03_13_0_c05_20201222_0002_S01_2,0
...,...,...
4693,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0005,3
4694,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0006,3
4695,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0007,3
4696,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0008,3


In [9]:
df = pd.concat([df_train, df_val], ignore_index=True)
df

,filename,class
0,V006_77_0_00_11_03_12_0_c01_20201013_0000_S01,0
1,V006_77_0_00_11_03_12_0_c01_20201013_0000_S01_1,0
2,V006_77_0_00_11_03_12_0_c01_20201013_0001_S01,0
3,V006_77_0_00_11_03_12_0_c01_20201013_0001_S01_1,0
4,V006_77_0_00_11_03_12_0_c01_20201013_0002_S01,0
...,...,...
28195,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0005,3
28196,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0006,3
28197,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0007,3
28198,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0008,3


In [11]:
import numpy as np

# 클래스별 인덱스 가져오기
class_indices = {}
for class_label in df['class'].unique():
    indices = df[df['class'] == class_label].index.tolist()  # 인덱스를 리스트로 변환
    class_indices[class_label] = indices

# 클래스별 인덱스를 섞은 후 비율에 맞게 분할
train_indices = []
val_indices = []
test_indices = []
for class_label, indices in class_indices.items():
    np.random.shuffle(indices)  # 클래스별 인덱스 섞기

    num_samples = len(indices)
    num_train = int(num_samples * 0.8)  # train 비율
    num_val = int(num_samples * 0.1)  # validation 비율
    num_test = num_samples - num_train - num_val

    train_indices.extend(indices[:num_train])
    val_indices.extend(indices[num_train:num_train+num_val])
    test_indices.extend(indices[num_train+num_val:])
    
# 데이터프레임을 train, validation, test로 분할
train_df = df.loc[train_indices]
val_df = df.loc[val_indices]
test_df = df.loc[test_indices]

In [17]:
train_df['class'].value_counts()

2    10128
0     5088
1     4752
3     2592
Name: class, dtype: int64

In [18]:
val_df['class'].value_counts()

2    1266
0     636
1     594
3     324
Name: class, dtype: int64

In [19]:
test_df['class'].value_counts()

2    1266
0     636
1     594
3     324
Name: class, dtype: int64

In [27]:
train_df.to_csv('train_df.csv')
val_df.to_csv('val_df.csv')
test_df.to_csv('test_df.csv')

In [21]:
train_df

,filename,class
2562,V006_77_0_00_11_03_13_0_c07_20201202_0314_S01_1,0
1812,V006_77_0_00_11_03_13_0_c06_20201209_0074_S01_1,0
1044,V006_77_0_00_11_03_13_0_c01_20201013_0005_S01,0
1214,V006_77_0_00_11_03_13_0_c04_20201222_0034_S01_1,0
25228,V006_77_0_00_11_03_13_0_c19_20201222_0006_S01_1,0
...,...,...
22318,V006_77_1_19_11_03_12_3_2851q_20201210_3_a0006,3
22613,V006_77_1_19_11_03_12_3_4396b_20201210_3_a0001,3
23012,V006_77_1_19_11_03_12_3_7711b_20201118_30_a0000,3
12511,V006_77_1_18_11_03_13_3_8419w_20201016_233_a0009,3


- train_df -> images

In [26]:
source_dir = "D:/rawdata_disease_resize/house/train/images"
target_dir = "D:/rawdata_disease_resize_shuffle/house/train/images"

not_found = 0
for index, row in tqdm(train_df.iterrows()):
    filename = row['filename']
    source_files = glob.glob(os.path.join(source_dir, filename + ".*"))  # 파일 확장자 앞 부분과 일치하는 파일들 선택
    if len(source_files) > 0:
        target_file = os.path.join(target_dir, os.path.basename(source_files[0]))  # 대상 디렉토리에 복사될 파일 경로
        shutil.copy(source_files[0], target_file)
    else:
        not_found += 1

print(not_found)

3771


In [28]:
source_dir = "D:/rawdata_disease_resize/house/validation/images"
target_dir = "D:/rawdata_disease_resize_shuffle/house/train/images"

not_found = 0
for index, row in tqdm(train_df.iterrows()):
    filename = row['filename']
    source_files = glob.glob(os.path.join(source_dir, filename + ".*"))  # 파일 확장자 앞 부분과 일치하는 파일들 선택
    if len(source_files) > 0:
        target_file = os.path.join(target_dir, os.path.basename(source_files[0]))  # 대상 디렉토리에 복사될 파일 경로
        shutil.copy(source_files[0], target_file)
    else:
        not_found += 1

print(not_found)

22560it [10:03, 37.40it/s]

18789


- train_df -> labels

In [30]:
source_dir = "D:/rawdata_disease_resize/house/train/labels"
target_dir = "D:/rawdata_disease_resize_shuffle/house/train/labels"

not_found = 0
for index, row in tqdm(train_df.iterrows()):
    filename = row['filename']
    source_files = glob.glob(os.path.join(source_dir, filename + ".*"))  # 파일 확장자 앞 부분과 일치하는 파일들 선택
    if len(source_files) > 0:
        target_file = os.path.join(target_dir, os.path.basename(source_files[0]))  # 대상 디렉토리에 복사될 파일 경로
        shutil.copy(source_files[0], target_file)
    else:
        not_found += 1

print(not_found)

22560it [32:27, 11.58it/s]

3771


In [29]:
source_dir = "D:/rawdata_disease_resize/house/validation/labels"
target_dir = "D:/rawdata_disease_resize_shuffle/house/train/labels"

not_found = 0
for index, row in tqdm(train_df.iterrows()):
    filename = row['filename']
    source_files = glob.glob(os.path.join(source_dir, filename + ".*"))  # 파일 확장자 앞 부분과 일치하는 파일들 선택
    if len(source_files) > 0:
        target_file = os.path.join(target_dir, os.path.basename(source_files[0]))  # 대상 디렉토리에 복사될 파일 경로
        shutil.copy(source_files[0], target_file)
    else:
        not_found += 1

print(not_found)

22560it [06:40, 56.27it/s]

18789


- val_df -> images

In [31]:
source_dir = "D:/rawdata_disease_resize/house/train/images"
target_dir = "D:/rawdata_disease_resize_shuffle/house/validation/images"

not_found = 0
for index, row in tqdm(val_df.iterrows()):
    filename = row['filename']
    source_files = glob.glob(os.path.join(source_dir, filename + ".*"))  # 파일 확장자 앞 부분과 일치하는 파일들 선택
    if len(source_files) > 0:
        target_file = os.path.join(target_dir, os.path.basename(source_files[0]))  # 대상 디렉토리에 복사될 파일 경로
        shutil.copy(source_files[0], target_file)
    else:
        not_found += 1

print(not_found)

2820it [06:01,  7.80it/s]

474


In [32]:
source_dir = "D:/rawdata_disease_resize/house/validation/images"
target_dir = "D:/rawdata_disease_resize_shuffle/house/validation/images"

not_found = 0
for index, row in tqdm(val_df.iterrows()):
    filename = row['filename']
    source_files = glob.glob(os.path.join(source_dir, filename + ".*"))  # 파일 확장자 앞 부분과 일치하는 파일들 선택
    if len(source_files) > 0:
        target_file = os.path.join(target_dir, os.path.basename(source_files[0]))  # 대상 디렉토리에 복사될 파일 경로
        shutil.copy(source_files[0], target_file)
    else:
        not_found += 1

print(not_found)

2820it [01:12, 38.77it/s]

2346


- val_df -> labels

In [33]:
source_dir = "D:/rawdata_disease_resize/house/train/labels"
target_dir = "D:/rawdata_disease_resize_shuffle/house/validation/labels"

not_found = 0
for index, row in tqdm(val_df.iterrows()):
    filename = row['filename']
    source_files = glob.glob(os.path.join(source_dir, filename + ".*"))  # 파일 확장자 앞 부분과 일치하는 파일들 선택
    if len(source_files) > 0:
        target_file = os.path.join(target_dir, os.path.basename(source_files[0]))  # 대상 디렉토리에 복사될 파일 경로
        shutil.copy(source_files[0], target_file)
    else:
        not_found += 1

print(not_found)

2820it [03:29, 13.44it/s]

474


In [34]:
source_dir = "D:/rawdata_disease_resize/house/validation/labels"
target_dir = "D:/rawdata_disease_resize_shuffle/house/validation/labels"

not_found = 0
for index, row in tqdm(val_df.iterrows()):
    filename = row['filename']
    source_files = glob.glob(os.path.join(source_dir, filename + ".*"))  # 파일 확장자 앞 부분과 일치하는 파일들 선택
    if len(source_files) > 0:
        target_file = os.path.join(target_dir, os.path.basename(source_files[0]))  # 대상 디렉토리에 복사될 파일 경로
        shutil.copy(source_files[0], target_file)
    else:
        not_found += 1

print(not_found)

2820it [00:46, 60.81it/s]

2346


- test_df -> images

In [35]:
source_dir = "D:/rawdata_disease_resize/house/train/images"
target_dir = "D:/rawdata_disease_resize_shuffle/house/test/images"

not_found = 0
for index, row in tqdm(test_df.iterrows()):
    filename = row['filename']
    source_files = glob.glob(os.path.join(source_dir, filename + ".*"))  # 파일 확장자 앞 부분과 일치하는 파일들 선택
    if len(source_files) > 0:
        target_file = os.path.join(target_dir, os.path.basename(source_files[0]))  # 대상 디렉토리에 복사될 파일 경로
        shutil.copy(source_files[0], target_file)
    else:
        not_found += 1

print(not_found)

2820it [05:54,  7.97it/s]

453


In [36]:
source_dir = "D:/rawdata_disease_resize/house/validation/images"
target_dir = "D:/rawdata_disease_resize_shuffle/house/test/images"

not_found = 0
for index, row in tqdm(test_df.iterrows()):
    filename = row['filename']
    source_files = glob.glob(os.path.join(source_dir, filename + ".*"))  # 파일 확장자 앞 부분과 일치하는 파일들 선택
    if len(source_files) > 0:
        target_file = os.path.join(target_dir, os.path.basename(source_files[0]))  # 대상 디렉토리에 복사될 파일 경로
        shutil.copy(source_files[0], target_file)
    else:
        not_found += 1

print(not_found)

2820it [01:08, 40.93it/s]

2367


- test_df -> labels

In [37]:
source_dir = "D:/rawdata_disease_resize/house/train/labels"
target_dir = "D:/rawdata_disease_resize_shuffle/house/test/labels"

not_found = 0
for index, row in tqdm(test_df.iterrows()):
    filename = row['filename']
    source_files = glob.glob(os.path.join(source_dir, filename + ".*"))  # 파일 확장자 앞 부분과 일치하는 파일들 선택
    if len(source_files) > 0:
        target_file = os.path.join(target_dir, os.path.basename(source_files[0]))  # 대상 디렉토리에 복사될 파일 경로
        shutil.copy(source_files[0], target_file)
    else:
        not_found += 1

print(not_found)

2820it [03:28, 13.51it/s]

453


In [38]:
source_dir = "D:/rawdata_disease_resize/house/validation/labels"
target_dir = "D:/rawdata_disease_resize_shuffle/house/test/labels"

not_found = 0
for index, row in tqdm(test_df.iterrows()):
    filename = row['filename']
    source_files = glob.glob(os.path.join(source_dir, filename + ".*"))  # 파일 확장자 앞 부분과 일치하는 파일들 선택
    if len(source_files) > 0:
        target_file = os.path.join(target_dir, os.path.basename(source_files[0]))  # 대상 디렉토리에 복사될 파일 경로
        shutil.copy(source_files[0], target_file)
    else:
        not_found += 1

print(not_found)

2820it [00:44, 63.32it/s]

2367
